In [9]:
import pandas as pd
from pandas import read_excel

file_name = 'no_training.xlsx' 
target_data = read_excel(file_name)
target_data = target_data.fillna(0)

In [10]:
feature_cols = list(target_data.columns[1:-7])

X_all = target_data[feature_cols]

In [11]:
from sklearn.ensemble import RandomForestClassifier
import pickle

mfmodel = pickle.load(open('mfmodel.pkl', 'rb'))

prediction = mfmodel.predict(X_all)
sales_cols = list(target_data.columns[:-7])
result = target_data[sales_cols]
result["mf_sales_prediction"] = prediction
result = result[result.mf_sales_prediction == 1]
print(result.shape[0])
print(result[:1000])
result.to_excel("mf_sales_prediction.xlsx", index=False)

15
     Client  Sex  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  \
26       64  1.0   61      38         1       0.0       0.0        1.0   
64      154  0.0   28     151         1       0.0       0.0        0.0   
167     389  0.0   43     158         1       1.0      28.0        0.0   
308     766  0.0   32      95         1       0.0       0.0        0.0   
357     878  1.0    5     173         1       0.0      22.0        0.0   
404    1007  1.0   42     152         1       1.0       4.0        0.0   
405    1008  1.0   37     103         1       2.0       4.0        0.0   
427    1073  1.0    4       9         1       0.0       3.0        0.0   
438    1093  0.0   22     181         1       0.0       0.0        0.0   
492    1226  1.0   82     211         1       0.0       4.0        1.0   
495    1229  1.0   64     175         1       0.0       0.0        0.0   
572    1435  1.0   44     148         1       0.0       0.0        1.0   
573    1440  0.0   29      96      

/home/syanakieva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [12]:
result

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,mf_sales_prediction
26,64,1.0,61,38,1,0.0,0.0,1.0,0.0,0.0,...,1449.484643,535.714286,518.127857,393.035357,51.0,51.0,3.0,42.0,4.0,1
64,154,0.0,28,151,1,0.0,0.0,0.0,0.0,0.0,...,1478.473929,0.000000,1478.473929,0.000000,38.0,38.0,0.0,38.0,0.0,1
167,389,0.0,43,158,1,1.0,28.0,0.0,0.0,0.0,...,670.821429,642.857143,0.000000,26.428571,6.0,6.0,2.0,0.0,1.0,1
308,766,0.0,32,95,1,0.0,0.0,0.0,0.0,0.0,...,398.392857,0.000000,0.000000,396.000000,6.0,6.0,0.0,0.0,5.0,1
357,878,1.0,5,173,1,0.0,22.0,0.0,0.0,0.0,...,1071.428571,0.000000,0.000000,0.000000,2.0,2.0,0.0,0.0,0.0,1
404,1007,1.0,42,152,1,1.0,4.0,0.0,0.0,0.0,...,380.250000,178.571429,0.000000,198.285714,6.0,6.0,2.0,0.0,3.0,1
405,1008,1.0,37,103,1,2.0,4.0,0.0,0.0,0.0,...,2372.888214,0.000000,0.000000,2369.102500,11.0,11.0,0.0,0.0,9.0,1
427,1073,1.0,4,9,1,0.0,3.0,0.0,1.0,0.0,...,1438.168214,0.000000,1104.918214,566.214286,34.0,31.0,0.0,17.0,14.0,1
438,1093,0.0,22,181,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,1
492,1226,1.0,82,211,1,0.0,4.0,1.0,1.0,0.0,...,1678.608571,214.285714,333.074286,1005.750000,77.0,22.0,6.0,47.0,10.0,1
